<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Intro_to_NDVI_Map_Nigeria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ZO2E1ER9o9u5gEsgCfWSwNUQtQDnggWpvvF2BwK7-jQ&tc=b21O1GlcdVwmSHFZzeLefPguzzrluBo9YH9yCBvbASk&cc=_cJIyZGzR_XOxol0tjgin3Orge4HhtSYeucMnfabhDs

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXns1aq4hemHAqlIvopTw82TdKiFLC3oCYAgFbKe9I0zRGrP6wBj4OQ

Successfully saved authorization token.


In [2]:
#NDVI
countries = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')

In [7]:
Nigeria = countries.filter(ee.Filter.eq('country_na', 'Nigeria'))

In [19]:
#Load 2017 Landsat 8
landsat = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')\
.filterDate('2017-01-01', '2017-12-31')\
.filterBounds(Nigeria)\
.median()

# Compute NDVI
ndvi = landsat.normalizedDifference(['B5', 'B3'])

NDVI_Palette = '00FF00', 'FFFF00', '000000'

In [20]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[10.4864, 7.2261], zoom_start = 6)

# Display
my_map.add_ee_layer(ndvi.clip(Nigeria), {'min': -0.3, 'max': 0.5, 'palette': NDVI_Palette}, 'NDVI')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)